In [2]:
!pip install geopandas

In [3]:
!pip install imageio

In [4]:
!pip install laspy

In [5]:
#!pip install rasterio

In [6]:
!pip install folium

In [7]:
import json
import urllib.request as urllib
import pickle

In [8]:
pre = 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/'
trail ='/ept.json'

In [9]:
gdf_dict ={'name':[], 'year':[], 'path':[], 'geometry':[], 'bounds':[]}
counter = 0
with open('../data/filename.txt') as f:
    for line in f:
        line_ = line.strip('/\n')
        gdf_dict['name'].append(line_)
        year = line_.split('_')[-1]
        gdf_dict['year'].append(year)
        path = pre + line_ + trail
        gdf_dict['path'].append(path)
        try:
            j_res = json.load(urllib.urlopen(path))
            wkt = j_res['srs']['wkt']
            bounds = j_res['bounds']
        except:
            print('path: {}\ncounter: {}'.format(path, counter))
            wkt = None
            bounds = None
        counter += 1
        gdf_dict['geometry'].append(wkt)
        gdf_dict['bounds'].append(bounds)
        if counter%100 == 0:
            print('\nCounter: {}'.format(counter))


Counter: 100

Counter: 200

Counter: 300

Counter: 400

Counter: 500

Counter: 600

Counter: 700

Counter: 800

Counter: 900

Counter: 1000

Counter: 1100

Counter: 1200

Counter: 1300

Counter: 1400
path: https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_WA_Western_North_2016_LAS_2018/ept.json
counter: 1477
path: https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_WA_Western_South_2016_LAS_2018/ept.json
counter: 1478

Counter: 1500


In [10]:
out = open('gdf_dict.pkl','wb')

In [11]:
import pickle
pickle.dump(gdf_dict, out)
out.close()

In [12]:
gdf_dict

{'name': ['AK_BrooksCamp_2012',
  'AK_Coastal_2009',
  'AK_Fairbanks-NSBorough_2010',
  'AK_Juneau_2012',
  'AK_Kenai_2008',
  'AK_MatanuskaSusitna-Lot1_2011',
  'AK_MatanuskaSusitna-Lot2_2011',
  'AK_NomeCreek_2010',
  'AK_NorthSlope_B10_2018',
  'AK_NorthSlope_B11_2018',
  'AK_NorthSlope_B12_2018',
  'AK_NorthSlope_B13_2018',
  'AK_NorthSlope_B14_2018',
  'AK_NorthSlope_B1_2018',
  'AK_NorthSlope_B2_2018',
  'AK_NorthSlope_B3_2018',
  'AK_NorthSlope_B4_2018',
  'AK_NorthSlope_B5_2018',
  'AK_NorthSlope_B6_2018',
  'AK_NorthSlope_B7_2018',
  'AK_NorthSlope_B8_2018',
  'AK_NorthSlope_B9_2018',
  'AK_ValdezB_2007',
  'AK_Valdez_2007',
  'AK_YukonFlats_2009',
  'AL_25Co_TL_2017',
  'AL_BaldwinCo-East_2011',
  'AL_BaldwinCo-West_2011',
  'AL_BlountCo_2010',
  'AL_ColbertLauderdale_2011',
  'AL_NorthAL_2019',
  'AL_TriCounty_2010',
  'AR-TN_NRCS_CaneCreek_2010',
  'AR-TN_NRCS_Sharkey_2010',
  'ARRA-AK_EkluntaGlacier_2010',
  'ARRA-AL_MobileBay_2010',
  'ARRA-AL_TennesseeRiver_2010',
  'ARR

In [13]:
import geopandas as gpd
import pandas as pd

In [14]:
df = pd.DataFrame(gdf_dict)
df.head()

,name,year,path,geometry,bounds
0,AK_BrooksCamp_2012,2012,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","[-17347360, 8065364, -12414, -17321558, 809116..."
1,AK_Coastal_2009,2009,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","[-15730544, 10937407, -19027, -15691854, 10976..."
2,AK_Fairbanks-NSBorough_2010,2010,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","[-16471700, 9519129, -45314, -16381190, 960963..."
3,AK_Juneau_2012,2012,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","[-15014449, 8012267, -35030, -14943073, 808364..."
4,AK_Kenai_2008,2008,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","[-16906356, 8303726, -166851, -16570284, 86397..."


In [15]:
df.dropna(axis=0, inplace=True)

In [16]:
df.reset_index(drop=True, inplace=True)

In [17]:
from shapely.geometry import Polygon
polygons =[]
for bound in df['bounds']:
  MINX, MINY, MAXX, MAXY = [bound[0],bound[1], bound[3],bound[4]] 
  polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))
  polygons.append(polygon)

In [18]:
df['geometry'] = polygons


In [19]:
df['geometry']

0       POLYGON ((-17347360 8065364, -17347360 8091166...
1       POLYGON ((-15730544 10937407, -15730544 109760...
2       POLYGON ((-16471700 9519129, -16471700 9609639...
3       POLYGON ((-15014449 8012267, -15014449 8083643...
4       POLYGON ((-16906356 8303726, -16906356 8639798...
                              ...                        
1591    POLYGON ((-12129392 5198641, -12129392 5443581...
1592    POLYGON ((-12312348 5096916, -12312348 5378558...
1593    POLYGON ((-12316229 5151367, -12316229 5426835...
1594    POLYGON ((-12312230 5248567, -12312230 5521277...
1595    POLYGON ((-12412389 5204407, -12412389 5361807...
Name: geometry, Length: 1596, dtype: object

In [20]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:3857")

In [21]:

gdf.to_crs(epsg=4326, inplace=True)

In [22]:
geo_data = open('geo_data.pkl','wb')

In [23]:
pickle.dump(gdf, geo_data)
geo_data.close()

In [24]:
MINXx, MINYy, MAXXx, MAXYy = [-93.756155, 41.918015, -93.747334, 41.921429]
poly = Polygon(((MINXx, MINYy), (MINXx, MAXYy), (MAXXx, MAXYy), (MAXXx, MINYy), (MINXx, MINYy)))
grid = gpd.GeoDataFrame([poly], columns=["geometry"])
grid.set_crs(epsg=4326, inplace=True)

,geometry
0,"POLYGON ((-93.75616 41.91801, -93.75616 41.921..."


In [25]:
gdf[gdf['geometry'].contains(grid['geometry'][0])]

,name,year,path,geometry,bounds
227,IA_FullState,FullState,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-96.64143 39.49761, -96.64143 44.333...","[-10758075, 4793202, -361911, -10034133, 55171..."
456,NE_Rainwater-2_2009,2009,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-105.53843 35.37498, -105.53843 45.5...","[-11748484, 4214957, -434, -10260270, 5703171,..."
595,SD_James_River_NRCS_J_2010,2010,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-103.21732 39.44034, -103.21732 48.1...","[-11490100, 4784944, 399, -10147792, 6127252, ..."
599,SD_McCook_County_2012,2012,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-103.39777 39.06858, -103.39777 47.8...","[-11510187, 4731500, -59, -10167535, 6074152, ..."
609,SD_Spink_County_2012,2012,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-136.78478 11.88771, -136.78478 66.0...","[-15226812, 1332932, -1726, -6668332, 9891412,..."
779,USGS_LPC_FL_Lower_Choctawhatchee_2017_LAS_2019,2019,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-102.81429 14.58351, -102.81429 43.8...","[-11445234, 1641247, -3808024, -7636576, 54499..."
1213,USGS_LPC_NE_Hat_White_Cherry_UTM14_2016_LAS_2019,2019,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-109.84370 35.16163, -109.84370 49.3...","[-12227745, 4185868, -2146839, -10071999, 6341..."
1219,USGS_LPC_NE_NRCS_OrthoLidar_B2_2017_LAS_2019,2019,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-109.93998 32.77236, -109.93998 47.4...","[-12238463, 3865128, -2146808, -10090237, 6013..."


In [26]:
for idx, bound in enumerate(gdf['geometry']):
  if bound.contains(poly):
    print(idx)

227
456
595
599
609
779
1213
1219


In [27]:
gdf.columns

Index(['name', 'year', 'path', 'geometry', 'bounds'], dtype='object')

In [49]:
gdf.head(4)

,name,year,path,geometry,bounds,area,centroid,envelope
0,AK_BrooksCamp_2012,2012,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-155.83399 58.46370, -155.83399 58.5...","[-17347360, 8065364, -12414, -17321558, 809116...",0.028051,POINT (-155.71809 58.52421),"POLYGON ((-155.83399 58.46370, -155.60220 58.4..."
1,AK_Coastal_2009,2009,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-141.30988 69.59264, -141.30988 69.7...","[-15730544, 10937407, -19027, -15691854, 10976...",0.042001,POINT (-141.13610 69.65306),"POLYGON ((-141.30988 69.59264, -140.96232 69.5..."
2,AK_Fairbanks-NSBorough_2010,2010,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-147.96780 64.65904, -147.96780 65.0...","[-16471700, 9519129, -45314, -16381190, 960963...",0.281134,POINT (-147.56127 64.83193),"POLYGON ((-147.96780 64.65904, -147.15473 64.6..."
3,AK_Juneau_2012,2012,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-134.87709 58.21334, -134.87709 58.5...","[-15014449, 8012267, -35030, -14943073, 808364...",0.215529,POINT (-134.55650 58.38141),"POLYGON ((-134.87709 58.21334, -134.23591 58.2..."


In [50]:
gdf.to_csv("../data/meta_data.csv", index=False)

In [51]:
gdf.shape

(1596, 8)

In [52]:
from osgeo import gdal
import matplotlib.pyplot as plt
# plot the shapefile
plt.rcParams['figure.figsize'] = 10,5
gdf.plot(color="white", edgecolor="#2e3131", figsize=(10, 5))
plt.title('USA: regions')

Text(0.5, 1.0, 'USA: regions')

In [53]:
print(gdf.crs)

EPSG:4326


In [54]:
gdf.crs = "EPSG:4326"

In [58]:
gdf.set_crs(epsg=4326, inplace=True)

,name,year,path,geometry,bounds,area,centroid,envelope
0,AK_BrooksCamp_2012,2012,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-155.83399 58.46370, -155.83399 58.5...","[-17347360, 8065364, -12414, -17321558, 809116...",0.028051,POINT (-155.71809 58.52421),"POLYGON ((-155.83399 58.46370, -155.60220 58.4..."
1,AK_Coastal_2009,2009,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-141.30988 69.59264, -141.30988 69.7...","[-15730544, 10937407, -19027, -15691854, 10976...",0.042001,POINT (-141.13610 69.65306),"POLYGON ((-141.30988 69.59264, -140.96232 69.5..."
2,AK_Fairbanks-NSBorough_2010,2010,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-147.96780 64.65904, -147.96780 65.0...","[-16471700, 9519129, -45314, -16381190, 960963...",0.281134,POINT (-147.56127 64.83193),"POLYGON ((-147.96780 64.65904, -147.15473 64.6..."
3,AK_Juneau_2012,2012,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-134.87709 58.21334, -134.87709 58.5...","[-15014449, 8012267, -35030, -14943073, 808364...",0.215529,POINT (-134.55650 58.38141),"POLYGON ((-134.87709 58.21334, -134.23591 58.2..."
4,AK_Kenai_2008,2008,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-151.87238 59.56594, -151.87238 61.0...","[-16906356, 8303726, -166851, -16570284, 86397...",4.512984,POINT (-150.36289 60.31337),"POLYGON ((-151.87238 59.56594, -148.85339 59.5..."
...,...,...,...,...,...,...,...,...
1591,WY_FEMA_East_B6_2019,2019,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-108.96018 42.25092, -108.96018 43.8...","[-12129392, 5198641, -120422, -11884452, 54435...",3.537350,POINT (-107.86002 43.05474),"POLYGON ((-108.96018 42.25092, -106.75985 42.2..."
1592,WY_FEMA_East_B7_2019,2019,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-110.60370 41.57089, -110.60370 43.4...","[-12312348, 5096916, -138094, -12030706, 53785...",4.718547,POINT (-109.33869 42.50340),"POLYGON ((-110.60370 41.57089, -108.07367 41.5..."
1593,WY_FEMA_East_B8_2019,2019,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-110.63857 41.93580, -110.63857 43.7...","[-12316229, 5151367, -134864, -12040761, 54268...",4.489370,POINT (-109.40128 42.84290),"POLYGON ((-110.63857 41.93580, -108.16400 41.9..."
1594,WY_FEMA_East_B9_2019,2019,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"POLYGON ((-110.60264 42.58203, -110.60264 44.3...","[-12312230, 5248567, -133600, -12039520, 55212...",4.354945,POINT (-109.37775 43.47087),"POLYGON ((-110.60264 42.58203, -108.15285 42.5..."


In [60]:
gdf['area'] = gdf.area
# Returns a GeoSeries of points for each geometric centroid.
gdf['centroid'] = gdf.centroid
# Returns a GeoSeries of geometries representing the point or smallest rectangular
# polygon (with sides parallel to the coordinate axes) that contains each object.
gdf['envelope'] = gdf.envelope

<ipython-input-60-e2d1ff1af52f>:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf.area
<ipython-input-60-e2d1ff1af52f>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.centroid


geo_data: geopandas data frame holding information of geographic geometry data: data frame containing values to be used for plotting the choropleth maps columns: the names of two columns — first column to be used as the key (‘NAME_2′ containing the names of the district in our case); second column are the values to be mapped (‘area’) key_on: Variable in geo_data to bind the data to. It must start with ‘feature’ fill_color: Can pass a hex code, color name, or one of these color palettes: ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’ fill_opacity: Opacity level, range 0–1 line_opacity: Geopath line color (boundary of the polygon) legend_name: Title for data legend

In [61]:
usa_g = gdf.drop(['centroid', 'envelope'], axis=1)

In [63]:
import folium
# Estimate centroids of the country
# --------------------
cent_x =gdf['geometry'].centroid.x.mean()
cent_y =gdf['geometry'].centroid.y.mean()

# Create a map object using Folium
# --------------------
map_usa = folium.Map(location=[cent_y, cent_x], zoom_start=7, tiles='OpenStreetMap')  

# Create the choropleth map
# --------------------
choro = folium.Choropleth(geo_data=usa_g,
                         name='choropleth',     
                          data=usa_g,
                          columns=['name', 'area'], 
#                           key_on='feature.NAME_2', 
                          fill_color='YlOrRd',
                          fill_opacity=0.6, 
                          line_opacity=0.8,
                          legend_name= "Area size across USA's subregion"
                          ).add_to(map_usa) 

# add labels to map
choro.geojson.add_child(folium.features.GeoJsonTooltip(fields=['name', 'area'],
                                                       aliases=['County', 'Mass'], 
                                                       labels = False))
folium.LayerControl().add_to(map_usa)


map_usa

<ipython-input-63-fcea7ba143b3>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent_x =gdf['geometry'].centroid.x.mean()
<ipython-input-63-fcea7ba143b3>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent_y =gdf['geometry'].centroid.y.mean()


In [64]:
# Estimate centroids of the country
# --------------------
cent_x =gdf['geometry'].centroid.x.mean()
cent_y =gdf['geometry'].centroid.y.mean()

# Create a map object using Folium
# --------------------
map_usa = folium.Map(location=[cent_y, cent_x], zoom_start=7, tiles='OpenStreetMap')  

# Create the choropleth map
# --------------------
choro = folium.Choropleth(geo_data=usa_g,
                         name='choropleth',     
                          data=usa_g,
                          columns=['name', 'area'], 
#                           key_on='feature.NAME_2', 
                          fill_color='YlOrRd',
                          fill_opacity=0.6, 
                          line_opacity=0.8,
                          legend_name= "Area size across USA's subregion"
                          ).add_to(map_usa) 

# add labels to map
choro.geojson.add_child(folium.features.GeoJsonTooltip(fields=['name', 'area'],
                                                       aliases=['County', 'Mass'], 
                                                       labels = True))
folium.LayerControl().add_to(map_usa)


map_usa

<ipython-input-64-6e60d58949e4>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent_x =gdf['geometry'].centroid.x.mean()
<ipython-input-64-6e60d58949e4>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent_y =gdf['geometry'].centroid.y.mean()
